Environment: napari_sparrow

In [ ]:
%load_ext autoreload
%autoreload 2

### Import packages

In [ ]:
import napari_sparrow as nas
import sparrow as sp
import os
import spatialdata as sd
import numpy as np
import cv2 as cv2
from skimage import io
import geopandas as gpd
from shapely.geometry import Point
import spatialdata as sd
import pandas as pd
from sparrow.table._table import _back_sdata_table_to_zarr
from matplotlib import pyplot as plt
import seaborn as sns
from skimage.measure import regionprops_table

### Set paths and create folders

In [ ]:
# Specify path to root folder
root_folder = 'D:/Data/2023-07-CarolineAsselman-FIm/Analysis_v2'

# Specify ROI name
ROI = 'A1_ROI1'

# Path to input folders
images_path = os.path.join(root_folder, 'cropped_images', ROI)
regions_path = os.path.join(root_folder, 'Qupath_annotations', ROI)
masks_path = os.path.join(root_folder, 'output', ROI, 'segmentation/masks')
ilastik_path = os.path.join(root_folder, 'output/ilastik')

# Path to output folders
output_path = os.path.join(root_folder, 'output', ROI, 'downstream_analysis')
plots_path = os.path.join(output_path, 'plots')
tables_path = os.path.join(output_path, 'tables')
os.makedirs(output_path, exist_ok = True)
os.makedirs(plots_path, exist_ok = True)
os.makedirs(tables_path, exist_ok = True)

### Specify channels to read in

In [ ]:
from gene_lists import keep, omitted_channels, immune

In [ ]:
input_list = []
c_coords_list = []

def create_input_lists(channel_list):
    for file in os.listdir(images_path):
        if (file.endswith('.tif') or file.endswith('tiff')) and any(channel in file for channel in channel_list):
            input_list.append(os.path.join(images_path, file))
            filename = os.path.splitext(file)[0]
            channel_name = [channel for channel in channel_list if channel in filename]
            c_coords_list.append(' '.join(channel_name))
        
create_input_lists(keep+omitted_channels)

### Create or read sdata

In [ ]:
sdata = sp.io.create_sdata(
    input= input_list,
    c_coords= c_coords_list,
    output_path=os.path.join(output_path, "sdata.zarr"),
    img_layer="raw_image",
    chunks=1024)
sdata

In [ ]:
# sdata = sd.read_zarr(os.path.join(output_path, "sdata.zarr"))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image', 
    channel = 'DAPI', 
    output=os.path.join(plots_path , 'DAPI.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.3,
    figsize=(35,22.5))

### Add region annotations

In [ ]:
gdf_regions = gpd.read_file(os.path.join(regions_path, 'regions.geojson'))
sdata = sp.sh._add_shapes_layer(sdata, input=gdf_regions, output_layer='regions', overwrite=True)

gdf_regions_grouped = gdf_regions.groupby('name')
for name, group in gdf_regions_grouped:
    sdata = sp.sh._add_shapes_layer(sdata, input=group, output_layer=f'regions_{name}', overwrite=True)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image', 
    channel = 'DAPI', 
    shapes_layer = 'regions',
    output=os.path.join(plots_path , 'DAPI_regions.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.3,
    figsize=(35,22.5))

In [ ]:
for name, group in gdf_regions_grouped:    
    sp.pl.plot_shapes(
        sdata, 
        img_layer='raw_image', 
        channel = 'DAPI', 
        shapes_layer = f'regions_{name}',
        output=os.path.join(plots_path , f'DAPI_{name}.png'),
        vmin_img=0,
        vmax_img=65535,
        alpha=0.3,
        figsize=(35,22.5))

### Add artifacts annotation

In [ ]:
gdf_artifacts = gpd.read_file(os.path.join(regions_path, 'artifacts.geojson'))
sdata = sp.sh._add_shapes_layer(sdata, input=gdf_artifacts, output_layer='artifacts', overwrite=True)

gdf_artifacts_grouped = gdf_artifacts.groupby('name')
for name, group in gdf_artifacts_grouped:
    sdata = sp.sh._add_shapes_layer(sdata, input=group, output_layer=f'artifacts_{name}', overwrite=True)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image', 
    channel = 'DAPI', 
    shapes_layer = 'artifacts',
    output=os.path.join(plots_path , 'DAPI_artifacts.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.3,
    figsize=(35,22.5))

In [ ]:
for name, group in gdf_artifacts_grouped:    
    sp.pl.plot_shapes(
        sdata, 
        img_layer='raw_image', 
        channel = 'DAPI', 
        shapes_layer = f'artifacts_{name}',
        output=os.path.join(plots_path , f'DAPI_{name}.png'),
        vmin_img=0,
        vmax_img=65535,
        alpha=0.3,
        figsize=(35,22.5))

### Add segmentation masks

In [ ]:
merged_mask = io.imread(os.path.join(masks_path, 'segmentation_mask_merged.tiff'))
nucleus_mask = io.imread(os.path.join(masks_path, 'segmentation_mask_nucleus.tiff'))
expanded_nucleus_mask = io.imread(os.path.join(masks_path, 'segmentation_mask_expanded_nucleus.tiff'))
immune_mask = io.imread(os.path.join(masks_path, 'segmentation_mask_immune.tiff'))
RBC_mask = io.imread(os.path.join(masks_path, 'segmentation_mask_RBC.tiff'))

In [ ]:
sdata = sp.im._add_label_layer(sdata, arr=merged_mask, chunks=1024, output_layer="segmentation_mask_merged")
sdata = sp.im._add_label_layer(sdata, arr=nucleus_mask, chunks=1024, output_layer="segmentation_mask_nucleus")
sdata = sp.im._add_label_layer(sdata, arr=expanded_nucleus_mask, chunks=1024, output_layer="segmentation_mask_expanded_nucleus")
sdata = sp.im._add_label_layer(sdata, arr=immune_mask, chunks=1024, output_layer="segmentation_mask_immune")
sdata = sp.im._add_label_layer(sdata, arr=RBC_mask, chunks=1024, output_layer="segmentation_mask_RBC")

In [ ]:
sdata = sp.sh._add_shapes_layer(sdata, input=sdata["segmentation_mask_merged"].data, output_layer="segmentation_mask_merged_boundaries", transformation=None, overwrite=True)
sdata = sp.sh._add_shapes_layer(sdata, input=sdata["segmentation_mask_nucleus"].data, output_layer="segmentation_mask_nucleus_boundaries", transformation=None, overwrite=True)
sdata = sp.sh._add_shapes_layer(sdata, input=sdata["segmentation_mask_expanded_nucleus"].data, output_layer="segmentation_mask_expanded_nucleus_boundaries", transformation=None, overwrite=True)
sdata = sp.sh._add_shapes_layer(sdata, input=sdata["segmentation_mask_immune"].data, output_layer="segmentation_mask_immune_boundaries", transformation=None, overwrite=True)
sdata = sp.sh._add_shapes_layer(sdata, input=sdata["segmentation_mask_RBC"].data, output_layer="segmentation_mask_RBC_boundaries", transformation=None, overwrite=True)

### Check sdata

In [ ]:
sdata

In [ ]:
sdata['raw_image'].c

In [ ]:
from napari_spatialdata import Interactive
Interactive(sdata)

### Intensity allocation

In [ ]:
sdata = sp.tb.allocate_intensity(
        sdata, img_layer="raw_image", labels_layer="segmentation_mask_merged", channels=keep+omitted_channels, chunks=4000, append=False, append_labels_layer_name=False)

In [ ]:
# Remove index 0
table = sdata.table[sdata.table.obs.index != '0']
del sdata.table
sdata.table = sd.models.TableModel.parse(table)

### Regionprops

In [ ]:
sdata = sp.tb.add_regionprop_features(
    sdata, labels_layer="segmentation_mask_merged", append_labels_layer_name=False)

sdata = sp.tb.add_regionprop_features(
    sdata, labels_layer="segmentation_mask_nucleus", append_labels_layer_name=True)

### Clean sdata.table.obs

In [ ]:
sdata.table.obs = (
    sdata.table.obs
    # Strip underscores from column names
    .rename(columns=lambda x: x.lstrip('_'))
    # Change names of centroid columns
    .rename(columns={
        'centroid-0': 'centroid_y',
        'centroid-1': 'centroid_x', 
        'centroid-0_nucleus': 'centroid_y_nucleus', 
        'centroid-1_nucleus': 'centroid_x_nucleus'
    })
    # Change names of nucleus columns
    .rename(columns=lambda x: x.replace('_segmentation_mask_nucleus', '_nucleus'))
)
_back_sdata_table_to_zarr(sdata=sdata)

### Determine whether cell has nucleus

In [ ]:
sdata.table.obs['nucleus'] = sdata.table.obs['area_nucleus'] > 0
_back_sdata_table_to_zarr(sdata=sdata)

In [ ]:
sdata.table.obs['nucleus'].value_counts()

### Assign cells to regions

In [ ]:
# Define function to assign cells to regions
def assign_region(centroid, gdf):
    for index, row in gdf.iterrows():
        if Point(centroid).within(row['geometry']):
            return row['name']
    return 'Not_assigned'

# Define function to calculate distance to region
def calculate_distance(centroid, gdf):
    return gdf.distance(Point(centroid)).min()

In [ ]:
# Assign cells
sdata.table.obs = (
    sdata.table.obs
    # Assign for each cell centroid in which region it occurs
    .assign(
        region_annotation = lambda x: x.apply(
            lambda row: assign_region((row['centroid_x'], row['centroid_y']), sdata.shapes['regions']), axis=1))
    # Assign for each cell centroid whether it occurs in an artifact
    .assign(
        artifact_annotation = lambda x: x.apply(
            lambda row: assign_region((row['centroid_x'], row['centroid_y']), sdata.shapes['artifacts']), axis=1))
    # Calculate distance to edge of lumen
    .assign(
        distance_to_lumen = lambda x: x.apply(
            lambda row: calculate_distance((row['centroid_x'], row['centroid_y']), sdata.shapes['regions_lumen']), axis=1))
)
_back_sdata_table_to_zarr(sdata=sdata)

In [ ]:
sdata.table.obs['region_annotation'].value_counts()

In [ ]:
sdata.table.obs['artifact_annotation'].value_counts()

### Add original segmentation mask labels and names

In [ ]:
from sparrow.image.segmentation._utils import _mask_to_original
df_masks=_mask_to_original(sdata, label_layer="segmentation_mask_merged", original_labels_layers=[ "segmentation_mask_immune", "segmentation_mask_RBC",  "segmentation_mask_expanded_nucleus"] )
df_masks['original_segmentation_mask'] = df_masks.idxmax(axis=1)
sdata.table.obs = sdata.table.obs.merge(df_masks, left_index=True, right_index=True, how='inner')
_back_sdata_table_to_zarr(sdata=sdata)

In [ ]:
pivot_table = pd.pivot_table(
    sdata.table.obs,
    index='original_segmentation_mask', 
    values='nucleus', 
    aggfunc=['count', lambda x: x.sum()])
pivot_table.columns = ['cell_count', 'nucleus_count']
pivot_table

### Calculate average intensity and add to sdata.table.obs

In [ ]:
# Add columns with total intensities to obs
intensity_df = sdata.table.to_df()
sdata.table.obs = pd.merge(sdata.table.obs, intensity_df, left_index=True, right_index=True, how="left")

# Correct intensities for area
for channel in keep+omitted_channels:
    sdata.table.obs[f'{channel}_average_intensity'] = sdata.table.obs[channel] / sdata.table.obs['area']

# Drop columns with total intensities
columns_to_remove = [f"{channel}" for channel in keep+omitted_channels]
sdata.table.obs = sdata.table.obs.drop(columns=columns_to_remove)

_back_sdata_table_to_zarr(sdata=sdata)

### Some plots of categorical variables

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'merged_masks_0_segmentation_masks.png'),
    alpha=1,
    cmap='rainbow',
    column='original_segmentation_mask',
    figsize=(35,22.5)
)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'merged_masks_1_nucleus.png'),
    alpha=1,
    cmap='rainbow',
    column='nucleus',
    figsize=(35,22.5)
)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'merged_masks_3_artifact_annotation.png'),
    alpha=1,
    cmap='Paired',
    column='artifact_annotation',
    figsize=(35,22.5)
)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='raw_image',
    channel = 'DAPI',
    shapes_layer="segmentation_mask_merged_boundaries", 
    output=os.path.join(plots_path , 'merged_masks_4_region_annotation.png'),
    alpha=1,
    cmap='rainbow',
    column='region_annotation',
    figsize=(35,22.5)
)

### Plots of average intensity

In [ ]:
average_intensity_path = os.path.join(plots_path, "average_intensity_plots_all")
os.makedirs(average_intensity_path, exist_ok = True)

for channel in keep+omitted_channels:
    sp.pl.plot_shapes(
        sdata, 
        img_layer='raw_image',
        channel = 'DAPI',
        shapes_layer="segmentation_mask_merged_boundaries", 
        output=os.path.join(average_intensity_path , f'average_intensity_{channel}.png'),
        alpha=1,
        cmap='viridis',
        column=f"{channel}_average_intensity",
        figsize=(35,22.5)
)

### Immune cells analysis

##### Incorporating ilastik object classifiers and create plots

In [ ]:
from scipy.spatial.distance import cdist
import glob

def preprocess_ilastik_csv(sdata, ilastik_csv_path):
    # Read and filter ilastik table
    ilastik_df = pd.read_csv(ilastik_csv_path)
    ilastik_df = ilastik_df[ilastik_df['Size in pixels'] >= sdata.table.obs['area'].min()] # This step attempts to remove all erroneous small objects created by ilastik that do no match to cell objects
    
    # Renaming columns
    ilastik_df.rename(columns={
        'User Label': 'ilastik_user_label',
        'Predicted Class': 'ilastik_predicted_class',
        'Center of the object_0': 'ilastik_centroid_x',
        'Center of the object_1': 'ilastik_centroid_y'
    }, inplace=True)

    # Renaming probability columns
    probability_columns = [col for col in ilastik_df.columns if col.startswith('Probability of')]
    for col in probability_columns:
        new_col_name = f'ilastik_probability_{col.split(" ")[-1]}'
        ilastik_df.rename(columns={col: new_col_name}, inplace=True)

    # Selecting the columns to keep
    columns_to_keep = [
        'ilastik_user_label',
        'ilastik_predicted_class',
        'ilastik_centroid_x',
        'ilastik_centroid_y'
    ] + [col for col in ilastik_df.columns if col.startswith('ilastik_probability_')]

    ilastik_df = ilastik_df[columns_to_keep]
    
    # Rename cells without user labels
    class_names = ilastik_df['ilastik_predicted_class'].unique()
    
    for index, row in ilastik_df.iterrows():
        ilastik_user_label = row['ilastik_user_label']
        if ilastik_user_label not in class_names:
            ilastik_df.at[index, 'ilastik_user_label'] = 'no_user_label'
    
    # Get ilastik nickname from file name
    filename = os.path.basename(ilastik_csv_path)
    filename_no_ext = os.path.splitext(filename)[0]
    
    if '_table' in filename_no_ext:
        csv_name = filename_no_ext.replace('_table', '')
    else:
        csv_name = filename_no_ext
    
    return ilastik_df, csv_name

def return_ilastik_data_for_nearest_cell(row, ilastik_df):
    # Extract coordinates
    adata_coordinates = (row['centroid_x'], row['centroid_y'])
    ilastik_coordinates = ilastik_df[['ilastik_centroid_x', 'ilastik_centroid_y']]

    # Calculate distances
    distances = cdist([adata_coordinates], ilastik_coordinates)[0]

    # Find nearest neighbor
    nearest_neighbor_index = np.argmin(distances)
    nearest_neighbor = ilastik_df.iloc[nearest_neighbor_index]

    return nearest_neighbor

def add_ilastik_to_sdata(sdata, ilastik_df, suffix=None):
    # Match ilastik objects to sdata objects
    merged_data = sdata.table.obs.apply(
        return_ilastik_data_for_nearest_cell,
        axis = 1, 
        ilastik_df = ilastik_df)
    merged_data = merged_data.drop(columns=['ilastik_centroid_x', 'ilastik_centroid_y'])
    if suffix is not None:
        merged_data.columns = [col + '_' + suffix for col in merged_data.columns]

    # Add ilastik data to sdata.table.obs
    table = sdata.table.obs
    for col in merged_data.columns:
        table[col] = merged_data[col]
    sdata.table.obs = table
    _back_sdata_table_to_zarr(sdata=sdata)

# Add data from all ilastik csv files in folder to sdata
ilastik_classifiers_data = {}

for ilastik_csv_path in glob.glob(f'{ilastik_path}/*_{ROI}_table.csv'):
    print(ilastik_csv_path)
    
    ilastik_df, csv_name = preprocess_ilastik_csv(sdata, ilastik_csv_path)
    csv_name = csv_name.replace(f'_{ROI}', '')
    ilastik_classifiers_data[csv_name] = ilastik_df

    add_ilastik_to_sdata(sdata, ilastik_df, suffix = csv_name)
    
# NOTE: This code will attempt to merge the data obtained from the ilastik classifiers to the sdata.tabel.obs based on the centroid coordinates of the sdata cells and the ilastik objects (i.e. for each cell in the sdata, the closest cell in the ilastik data will be considered a match). 

In [ ]:
# Create plots per ilastik classifier
for csv_name in ilastik_classifiers_data:
    
    plots_path_ilastik = os.path.join(plots_path, 'ilastik_classifiers', csv_name)
    os.makedirs(plots_path_ilastik, exist_ok=True)
    
    class_names = ['pos', 'neg']
    probability_columns = [col for col in sdata.table.obs.columns if col.startswith('ilastik_probability_') and any(col.endswith(f"_{name}_{csv_name}") for name in class_names)]
    
    for column in sdata.table.obs.columns:
        if column == f'ilastik_user_label_{csv_name}' or column == f'ilastik_predicted_class_{csv_name}':
            sp.pl.plot_shapes(
                sdata, 
                img_layer='raw_image', 
                channel='DAPI', 
                shapes_layer='segmentation_mask_merged_boundaries',
                output=f'{plots_path_ilastik}/{column}.png',
                alpha=1,
                cmap='rainbow',
                column=column,
                figsize=(35,22.5))
            
        if column in probability_columns:
            sp.pl.plot_shapes(
                sdata, 
                img_layer='raw_image', 
                channel='DAPI', 
                shapes_layer='segmentation_mask_merged_boundaries',
                output=f'{plots_path_ilastik}/{column}.png',
                alpha=1,
                cmap='viridis',
                column=column,
                figsize=(35,22.5))

### Save as csv

In [ ]:
df_cells = sdata.table.obs
df_cells.to_csv(os.path.join(tables_path, "cells_table.csv"), index=True)

In [ ]:
df_cells

### Save as anndata

In [ ]:
centroid_x = sdata.table.obs['centroid_x']
centroid_y = sdata.table.obs['centroid_y']
coordinates_array = np.column_stack((centroid_x, centroid_y))
sdata.table.obsm["spatial"] = coordinates_array

In [ ]:
sdata.table.write_h5ad(filename=os.path.join(tables_path, "anndata.h5ad"))